In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

# 데이터 전처리

In [2]:
df = pd.read_csv('noshow.csv')

## 나이가 0보다 작은 이상치 제거

In [68]:
df[df['Age'] < 0]

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
99832,4.659432e+14,5775010,F,2016-06-06T08:58:13Z,2016-06-06T00:00:00Z,-1,ROMÃO,0,0,0,0,0,0,No


In [69]:
df.set_index('AppointmentID', inplace = True)

In [70]:
df['PatientId'] = df['PatientId'].apply(str)

## data 11만건 / 환자 6만건 -> 환자 각각 한명의 pattern 을 분석하는것 무의미

In [71]:
df['PatientId'].drop_duplicates().shape[0] # 6만건 

62299

## 환자 코드 삭제

In [72]:
df.drop(['PatientId'],axis=1, inplace=True)

## No-show one hot encoding

In [73]:
df['No-show'] = df['No-show'].apply(lambda x : 1 if x == 'Yes' else 0)

## 약속 시간과 약속을 잡은 시간간의 격차 구하기 -> hour 기준

In [74]:
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])

In [75]:
df['AppointmentDay'] = df['AppointmentDay'] + timedelta(hours = 23, minutes = 59, seconds = 59)

In [76]:
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'])

In [77]:
df['TimeDifference'] = (df['AppointmentDay'] - df['ScheduledDay']).dt.days * 24 + (df['AppointmentDay'] - df['ScheduledDay']).dt.seconds//3600

## 예약한 날짜가 예약 날짜보다 앞인 경우 제거 (오염된 데이터)

In [78]:
df_ = df_[df_['TimeDifference'] > 0]

## 나이 -> 연령대 로 변경 

In [79]:
df_temp = df_
age = df_temp['Age']
for i in range(0, 130, 10):    #연령대 for 문 이용 한번에 전환 
    age = age.apply(lambda x : i if (x < i+10 and x >= i) else x)

In [80]:
df_['Age'] = age

## 요일 정보 추가

In [81]:
df_['DayofWeek'] =df_['AppointmentDay'].dt.dayofweek

In [82]:
df_.columns

Index(['Gender', 'ScheduledDay', 'AppointmentDay', 'TimeDifference', 'Age',
       'Neighbourhood', 'Scholarship', 'Hipertension', 'Diabetes',
       'Alcoholism', 'Handcap', 'SMS_received', 'No-show', 'DayofWeek'],
      dtype='object')

In [84]:
sec = df.loc[:,'Age': 'No-show']
first = df[['Gender', 'ScheduledDay','AppointmentDay', 'TimeDifference']]
df_ = pd.concat([first,df_['DayofWeek'],sec],axis=1)

In [86]:
df_.sort_values(by='AppointmentDay', ascending = True , inplace = True)

In [87]:
df_.head()

,Gender,ScheduledDay,AppointmentDay,TimeDifference,DayofWeek,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
AppointmentID,,,,,,,,,,,,,,
5640200,M,2016-04-29 10:14:47,2016-04-29 23:59:59,13,4.0,19,SANTO ANTÔNIO,0,0,0,0,0,0,0
5629303,F,2016-04-27 13:16:02,2016-04-29 23:59:59,58,4.0,49,CONSOLAÇÃO,0,0,0,0,0,0,0
5629304,F,2016-04-27 13:16:02,2016-04-29 23:59:59,58,4.0,49,CONSOLAÇÃO,0,0,0,0,0,0,0
5638519,F,2016-04-29 08:06:53,2016-04-29 23:59:59,15,4.0,0,ITARARÉ,0,0,0,0,0,0,0
5490079,M,2016-03-18 14:08:26,2016-04-29 23:59:59,1017,4.0,7,ILHA DE SANTA MARIA,0,0,0,0,0,0,1


df_.to_csv('/Users/Keith/Desktop/ds_pjt/noshow_final.csv')